In [70]:
#pip install wordcloud

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
LFIGSIZE=(2,8)
FIGSIZE=(14,8)
FIGSIZE2=(20,8)
FIGSIZE3=(30,12)


In [71]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import numpy as np
from wordcloud import WordCloud, STOPWORDS
import re
import time

pd.set_option('display.max_columns', None)

## Data process functions


In [72]:
def check_duplicates(df: pd.DataFrame):
    ''' 
    Print duplicated rows and return dataframe without duplicates
    '''

    duplicates = df[df.duplicated()]
    print("Doulicate Rows: ")
    print("===============================")
    print(duplicates)
    
    return df.drop_duplicates()

In [73]:
def cols_all_nan(df):
    ''' Returns columns with NaN for every row in df
    
    INPUT:
    df - pandas dataframe
    
    OUTPUT:
    list of column indices for all columns with all NaN values 
    '''
    return df.columns[df.isnull().all()]

def cols_with_nan(df, percentage):
    '''Returns the column names that have more than (percentage * 100) %
    of their values missing
    
    INPUT:
    df - pandas dataframe
    percentage - float, percentage of values missing

    OUTPUT:
    list of column indices for all columns where more than (percentage * 100) %
    of their values are missing
    '''
    return df.columns[np.sum(df.isnull())/df.shape[0] > percentage]

def cols_nan_rate(df):
    '''Returns the column names that have more than (percentage * 100) %
    of their values missing
    
    INPUT:
    df - pandas dataframe
    percentage - float, percentage of values missing

    OUTPUT:
    list of column indices for all columns where more than (percentage * 100) %
    of their values are missing
    '''
    for col in df.columns:
        
    return df.columns[np.sum(df.isnull())/df.shape[0] > percentage]

## Query functions

In [74]:
def query_vid(df: pd.DataFrame , id):
    '''
    Query with a VAERS id
    '''
    
    print(df[df['VAERS_ID']==id])
    
             

In [75]:
# read data
# read in all datasets
data_path_21= '/home/wwang/workspace/covid19_vax_analysis/data/2021VAERSData-08272021/'
data_path_20= '/home/wwang/workspace/covid19_vax_analysis/data/2020VAERSData-08272021/'

df_patients_21 = pd.read_csv(data_path_21 + '2021VAERSDATA.csv', engine='python',  encoding='latin1')
df_symptoms_21 = pd.read_csv(data_path_21 + '2021VAERSSYMPTOMS.csv', encoding='latin1')
df_vax_21 = pd.read_csv(data_path_21 + '2021VAERSVAX.csv', encoding='latin1')

df_patients_20 = pd.read_csv(data_path_20 +'2020VAERSDATA.csv', engine='python', encoding='latin1')
df_symptoms_20 = pd.read_csv(data_path_20 + '2020VAERSSYMPTOMS.csv', encoding='latin1')
df_vax_20 = pd.read_csv(data_path_20 + '2020VAERSVAX.csv', encoding='latin1')



In [76]:
df_patients = pd.concat([df_patients_20, df_patients_21])
df_symptoms = pd.concat([df_symptoms_20, df_symptoms_21])
df_vax = pd.concat([df_vax_20, df_vax_21])


In [77]:
#Study table dimension
print(df_patients_21.shape)
print(df_symptoms_21.shape)
print(df_vax_21.shape)

print(df_patients_20.shape)
print(df_symptoms_20.shape)
print(df_vax_20.shape)

print(df_patients.shape)
print(df_symptoms.shape)
print(df_vax.shape)

(484964, 35)
(648823, 11)
(505121, 8)
(49782, 35)
(61103, 11)
(59710, 8)
(534746, 35)
(709926, 11)
(564831, 8)


In [78]:
df_patients.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 534746 entries, 0 to 484963
Data columns (total 35 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   VAERS_ID      534746 non-null  int64  
 1   RECVDATE      534746 non-null  object 
 2   STATE         473364 non-null  object 
 3   AGE_YRS       483105 non-null  float64
 4   CAGE_YR       442918 non-null  float64
 5   CAGE_MO       4328 non-null    float64
 6   SEX           534746 non-null  object 
 7   RPT_DATE      1354 non-null    object 
 8   SYMPTOM_TEXT  534619 non-null  object 
 9   DIED          6476 non-null    object 
 10  DATEDIED      5808 non-null    object 
 11  L_THREAT      8348 non-null    object 
 12  ER_VISIT      226 non-null     object 
 13  HOSPITAL      30322 non-null   object 
 14  HOSPDAYS      20501 non-null   float64
 15  X_STAY        315 non-null     object 
 16  DISABLE       8563 non-null    object 
 17  RECOVD        485474 non-null  object 
 18  VAX_

In [114]:
df_patients.columns

Index(['VAERS_ID', 'RECVDATE', 'STATE', 'AGE_YRS', 'CAGE_YR', 'CAGE_MO', 'SEX',
       'RPT_DATE', 'SYMPTOM_TEXT', 'DIED', 'DATEDIED', 'L_THREAT', 'ER_VISIT',
       'HOSPITAL', 'HOSPDAYS', 'X_STAY', 'DISABLE', 'RECOVD', 'VAX_DATE',
       'ONSET_DATE', 'NUMDAYS', 'LAB_DATA', 'V_ADMINBY', 'V_FUNDBY',
       'OTHER_MEDS', 'CUR_ILL', 'HISTORY', 'PRIOR_VAX', 'SPLTTYPE',
       'FORM_VERS', 'TODAYS_DATE', 'BIRTH_DEFECT', 'OFC_VISIT', 'ER_ED_VISIT',
       'ALLERGIES'],
      dtype='object')

# Important Dates
* RECVDATE: The date the VAERS form information was received to our processing center.
* RPT_DATE: Date the VAERS form was completed by the reporter as recorded on the specified field of the form. This is a VAERS 1 form field only. 
* TODAYS_DATE: Date Form Completed.
* VAX_DATE
* ONSET_DATE

So, when one of the RPT_DATE, TODAYS_DATE are missing, it should use the other na value

In [139]:
df_patients.columns

Index(['VAERS_ID', 'RECVDATE', 'STATE', 'AGE_YRS', 'CAGE_YR', 'CAGE_MO', 'SEX',
       'RPT_DATE', 'SYMPTOM_TEXT', 'DIED', 'DATEDIED', 'L_THREAT', 'ER_VISIT',
       'HOSPITAL', 'HOSPDAYS', 'X_STAY', 'DISABLE', 'RECOVD', 'VAX_DATE',
       'ONSET_DATE', 'NUMDAYS', 'LAB_DATA', 'V_ADMINBY', 'V_FUNDBY',
       'OTHER_MEDS', 'CUR_ILL', 'HISTORY', 'PRIOR_VAX', 'SPLTTYPE',
       'FORM_VERS', 'TODAYS_DATE', 'BIRTH_DEFECT', 'OFC_VISIT', 'ER_ED_VISIT',
       'ALLERGIES'],
      dtype='object')

In [168]:
df_patients[df_patients['TODAYS_DATE'].isnull()].shape[0]

3913

In [169]:
df_patients[df_patients['RECVDATE'].isnull()].shape[0]

0

In [170]:
df_patients[df_patients['RPT_DATE'].isnull()].shape[0]

533392

In [172]:
df_patients[df_patients['VAX_DATE'].isnull()].shape[0]

33774

In [173]:
df_patients[df_patients['ONSET_DATE'].isnull()].shape[0]

35069

In [176]:
df_patients[df_patients['DATEDIED'].isnull() == False ].shape[0]

5808

In [177]:
df_patients[df_patients['DIED'] == 'Y' ].shape[0]

6476

In [171]:
df_patients.shape[0]

534746

In [151]:
df_patients[df_patients['TODAYS_DATE'].isnull()][['TODAYS_DATE','RPT_DATE']]

,TODAYS_DATE,RPT_DATE
67,NaN,12/26/2019
218,NaN,01/03/2020
220,NaN,01/03/2020
361,NaN,01/06/2020
427,NaN,12/31/2019
...,...,...
484513,NaN,NaN
484895,NaN,NaN
484896,NaN,NaN
484897,NaN,NaN


In [112]:

s=df_patients.groupby(['RECVDATE'])['VAERS_ID'].nunique()
daily_case_count= pd.DataFrame({'DATE': s.index, 'COUNT': s.values})
#daily_case_count.set_index(['RECVDATE'])
daily_case_count['DATE'] = daily_case_count['DATE'].astype('datetime64[ns]')
daily_case_count['DATE'] = daily_case_count['DATE'].dt.strftime("%Y/%m/%d")
daily_case_count =daily_case_count.sort_values(by='DATE')
daily_case_count.set_index('DATE', inplace=True)
daily_case_count
#weekly_case_count = daily_case_count.resample('w').sum()

,COUNT
DATE,
2020/01/01,23
2020/01/02,103
2020/01/03,95
2020/01/04,26
2020/01/05,32
...,...
2021/08/16,2808
2021/08/17,1292
2021/08/18,1536


In [98]:
daily_case_count

,VAERS_ID
RECVDATE,
01/01/2020,23
01/01/2021,514
01/02/2020,103
01/02/2021,582
01/03/2020,95
...,...
12/27/2020,303
12/28/2020,1084
12/29/2020,1848


,VAERS_ID,SYMPTOM1,SYMPTOMVERSION1,SYMPTOM2,SYMPTOMVERSION2,SYMPTOM3,SYMPTOMVERSION3,SYMPTOM4,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5
0,855017,Arthralgia,22.1,Chills,22.1,Injection site pain,22.1,Pyrexia,22.1,NaN,NaN
1,855018,Chills,22.1,Fatigue,22.1,Hypertension,22.1,Hypoaesthesia,22.1,Injected limb mobility decreased,22.1
2,855018,Muscular weakness,22.1,Pain in extremity,22.1,Pyrexia,22.1,Tremor,22.1,Vertigo,22.1
3,855019,Pain,22.1,Pruritus,22.1,Rash,22.1,NaN,NaN,NaN,NaN
4,855020,Chills,22.1,Influenza like illness,22.1,Myalgia,22.1,Pain in extremity,22.1,Pyrexia,22.1
...,...,...,...,...,...,...,...,...,...,...,...
648818,1590973,Laboratory test,24.0,Menstrual disorder,24.0,Premature menopause,24.0,NaN,NaN,NaN,NaN
648819,1590980,COVID-19,24.0,SARS-CoV-2 test positive,24.0,Vaccine breakthrough infection,24.0,NaN,NaN,NaN,NaN
648820,1590982,COVID-19,24.0,Chest X-ray abnormal,24.0,Cough,24.0,Decreased appetite,24.0,Dyspnoea,24.0
648821,1590982,Electrocardiogram abnormal,24.0,Fatigue,24.0,Olfactory nerve disorder,24.0,Pneumonia,24.0,Rhinorrhoea,24.0


In [115]:
df_patients = check_duplicates(df_patients)
df_symptoms = check_duplicates(df_symptoms)
df_vax = check_duplicates(df_vax)

Doulicate Rows: 
Empty DataFrame
Columns: [VAERS_ID, RECVDATE, STATE, AGE_YRS, CAGE_YR, CAGE_MO, SEX, RPT_DATE, SYMPTOM_TEXT, DIED, DATEDIED, L_THREAT, ER_VISIT, HOSPITAL, HOSPDAYS, X_STAY, DISABLE, RECOVD, VAX_DATE, ONSET_DATE, NUMDAYS, LAB_DATA, V_ADMINBY, V_FUNDBY, OTHER_MEDS, CUR_ILL, HISTORY, PRIOR_VAX, SPLTTYPE, FORM_VERS, TODAYS_DATE, BIRTH_DEFECT, OFC_VISIT, ER_ED_VISIT, ALLERGIES]
Index: []
Doulicate Rows: 
Empty DataFrame
Columns: [VAERS_ID, SYMPTOM1, SYMPTOMVERSION1, SYMPTOM2, SYMPTOMVERSION2, SYMPTOM3, SYMPTOMVERSION3, SYMPTOM4, SYMPTOMVERSION4, SYMPTOM5, SYMPTOMVERSION5]
Index: []
Doulicate Rows: 
        VAERS_ID VAX_TYPE                     VAX_MANU  VAX_LOT  \
17        855033      UNK         UNKNOWN MANUFACTURER  N010497   
43        855065     FLUX         UNKNOWN MANUFACTURER      NaN   
71        855107   VARZOS  GLAXOSMITHKLINE BIOLOGICALS      NaN   
3863      858939      UNK         UNKNOWN MANUFACTURER      NaN   
4085      860131      UNK         UNKNOWN MANUF

In [ ]:


print(df_patients.shape)
print(df_symptoms.shape)
print(df_vax.shape)

In [116]:
del df_patients_21
del df_symptoms_21
del df_vax_21
del df_patients_20
del df_symptoms_20
del df_vax_20

In [117]:
# Study table columns
print(df_patients.columns)
print(df_symptoms.columns)
print(df_vax.columns)

Index(['VAERS_ID', 'RECVDATE', 'STATE', 'AGE_YRS', 'CAGE_YR', 'CAGE_MO', 'SEX',
       'RPT_DATE', 'SYMPTOM_TEXT', 'DIED', 'DATEDIED', 'L_THREAT', 'ER_VISIT',
       'HOSPITAL', 'HOSPDAYS', 'X_STAY', 'DISABLE', 'RECOVD', 'VAX_DATE',
       'ONSET_DATE', 'NUMDAYS', 'LAB_DATA', 'V_ADMINBY', 'V_FUNDBY',
       'OTHER_MEDS', 'CUR_ILL', 'HISTORY', 'PRIOR_VAX', 'SPLTTYPE',
       'FORM_VERS', 'TODAYS_DATE', 'BIRTH_DEFECT', 'OFC_VISIT', 'ER_ED_VISIT',
       'ALLERGIES'],
      dtype='object')
Index(['VAERS_ID', 'SYMPTOM1', 'SYMPTOMVERSION1', 'SYMPTOM2',
       'SYMPTOMVERSION2', 'SYMPTOM3', 'SYMPTOMVERSION3', 'SYMPTOM4',
       'SYMPTOMVERSION4', 'SYMPTOM5', 'SYMPTOMVERSION5'],
      dtype='object')
Index(['VAERS_ID', 'VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES',
       'VAX_ROUTE', 'VAX_SITE', 'VAX_NAME'],
      dtype='object')


In [118]:
# check if all datasets have the same number of patients
#print(df_patients_21["VAERS_ID"].nunique())
#print(df_symptoms_21["VAERS_ID"].nunique())
#print(df_vax_21["VAERS_ID"].nunique())
#print(df_patients_20["VAERS_ID"].nunique())
#print(df_symptoms_20["VAERS_ID"].nunique())
#print(df_vax_20["VAERS_ID"].nunique())
print(df_patients["VAERS_ID"].nunique())
print(df_symptoms["VAERS_ID"].nunique())
print(df_vax["VAERS_ID"].nunique())



534746
534743
534746


In [ ]:
df_patients.head()

In [119]:
dirty_df_patients = df_patients
dirty_df_symptoms = df_symptoms
dirty_df_vax = df_vax

In [120]:
merged_vaers_data = dirty_df_patients.merge(dirty_df_vax, on = 'VAERS_ID', how = 'inner')
#merged_vaers_data = merged_vaers_data.merge(dirty_df_symptoms, on = 'VAERS_ID', how = 'inner')

In [143]:
merged_vaers_data[["VAERS_ID", "RECVDATE", "VAX_DATE","TODAYS_DATE"]]

,VAERS_ID,RECVDATE,VAX_DATE,TODAYS_DATE
0,855017,01/01/2020,12/30/2019,01/01/2020
1,855018,01/01/2020,12/30/2019,01/01/2020
2,855019,01/01/2020,12/23/2019,01/01/2020
3,855020,01/01/2020,12/01/2019,01/01/2020
4,855021,01/01/2020,12/31/2019,01/01/2020
...,...,...,...,...
563775,1590960,08/20/2021,03/31/2021,08/20/2021
563776,1590960,08/20/2021,03/31/2021,08/20/2021
563777,1590973,08/20/2021,12/21/2020,08/20/2021
563778,1590980,08/20/2021,02/17/2021,08/20/2021


In [137]:
merged_vaers_data_covid = merged_vaers_data[merged_vaers_data["VAX_TYPE"]=='COVID19']
print(merged_vaers_data_covid.head())
s=merged_vaers_data_covid.groupby(['RECVDATE'])['VAERS_ID'].nunique()
s
daily_case_count= pd.DataFrame({'DATE': s.index, 'COUNT': s.values})
#daily_case_count.set_index(['RECVDATE'])
#daily_case_count['DATE'] = daily_case_count['DATE'].astype('datetime64[ns]')
#daily_case_count['DATE'] = daily_case_count['DATE'].dt.strftime("%Y/%m/%d")
#daily_case_count =daily_case_count.sort_values(by='DATE')
#daily_case_count.set_index('DATE', inplace=True)
#daily_case_count
#weekly_case_count = daily_case_count.resample('w').sum()

       VAERS_ID    RECVDATE STATE  AGE_YRS  CAGE_YR  CAGE_MO SEX RPT_DATE  \
46358    902418  12/15/2020    NJ     56.0     56.0      NaN   F      NaN   
46377    902440  12/15/2020    AZ     35.0     35.0      NaN   F      NaN   
46384    902446  12/15/2020    WV     55.0     55.0      NaN   F      NaN   
46404    902464  12/15/2020    LA     42.0     42.0      NaN   M      NaN   
46405    902465  12/15/2020    AR     60.0     60.0      NaN   F      NaN   

                                            SYMPTOM_TEXT DIED DATEDIED  \
46358  Patient experienced mild numbness traveling fr...  NaN      NaN   
46377                                       C/O Headache  NaN      NaN   
46384  felt warm, hot and face and ears were red and ...  NaN      NaN   
46404  within 15 minutes progressive light-headedness...  NaN      NaN   
46405  Pt felt wave come over body @ 1218 starting in...  NaN      NaN   

      L_THREAT ER_VISIT HOSPITAL  HOSPDAYS X_STAY DISABLE RECOVD    VAX_DATE  \
46358      N

In [144]:
merged_vaers_data_covid[["VAERS_ID", "RECVDATE", "VAX_DATE","TODAYS_DATE"]]

,VAERS_ID,RECVDATE,VAX_DATE,TODAYS_DATE
46358,902418,12/15/2020,12/15/2020,12/15/2020
46377,902440,12/15/2020,12/15/2020,12/15/2020
46384,902446,12/15/2020,12/15/2020,12/15/2020
46404,902464,12/15/2020,12/15/2020,12/15/2020
46405,902465,12/15/2020,12/15/2020,12/15/2020
...,...,...,...,...
563775,1590960,08/20/2021,03/31/2021,08/20/2021
563776,1590960,08/20/2021,03/31/2021,08/20/2021
563777,1590973,08/20/2021,12/21/2020,08/20/2021
563778,1590980,08/20/2021,02/17/2021,08/20/2021


In [ ]:
print(df_patients.columns)
print(df_vax.columns)
print(df_symptoms.columns)
print(merged_vaers_data.columns)

print(df_patients.shape)
print(df_vax.shape)
print(df_symptoms.shape)
print(merged_vaers_data.shape)

In [ ]:
merged_vaers_data[merged_vaers_data.duplicated()]

In [ ]:
merged_vaers_data.loc[merged_vaers_data['VAERS_ID'] == 855065 ]

## Check How many different vaccines the merged table has

In [ ]:
merged_vaers_data['VAX_TYPE'].unique()

## Remove non-covid19 vaccine data

In [ ]:
# Remove none-covid19 vaccine data
merged_vaers_data =  merged_vaers_data[merged_vaers_data['VAX_TYPE'] == 'COVID19']


In [ ]:
merged_vaers_data.head()

In [ ]:
print(merged_vaers_data.shape)

In [ ]:
# count of unique value of each column
print(merged_vaers_data.nunique())

In [ ]:
#Count Occurences with Pandas value_counts()
#pd.set_option('display.max_rows', 40)
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
print(merged_vaers_data['VAERS_ID'].value_counts())

In [ ]:
df_patients[df_patients['VAERS_ID']==1019670]

In [ ]:
df_vax[df_vax['VAERS_ID']==1019670]

In [ ]:
df_symptoms[df_symptoms['VAERS_ID']==1019670].count()

In [ ]:
merged_vaers_data[merged_vaers_data['VAERS_ID']==1019670].count()

In [ ]:
merged_vaers_data.info()

In [ ]:
# Change text data to upper cases
merged_vaers_data= merged_vaers_data.applymap(lambda s:s.upper() if type(s) == str else s)

In [ ]:
merged_vaers_data['SYMPTOM_TEXT']

# Report functions



In [ ]:
def ana_disease(df : pd.DataFrame, disease : str, bins : int):
    NO_OF_BINS=bins
    df_disease = df[df.eq(disease.upper()).any(1)]
    #print(df_disease.head())
    print(df_disease['VAERS_ID'].duplicated())
    df_disease['AGE_YRS'].astype(float)
    fig = plt.figure()
    sns.set(font_scale = (2))
    plt.style.use('dark_background')
    ax0 = plt.subplot2grid((1,5), (0,0), colspan=1)
    ax1 = plt.subplot2grid((1,5), (0,1), colspan=4)
    df_disease['AGE_YRS'].plot(kind='box', figsize=LFIGSIZE, ax=ax0, boxprops=dict(linestyle='-', linewidth=4, color='r'), 
                    medianprops=dict(linestyle='-', linewidth=4),
                    whiskerprops=dict(linestyle='-', linewidth=4, color='g'), 
                    capprops=dict(linestyle='-', linewidth=4),)
    df_disease['AGE_YRS'].plot(kind='hist', bins=NO_OF_BINS, figsize=FIGSIZE, edgecolor='r', linewidth=2,ax=ax1)
    ax0.set_title('Average Age')
    ax1.set_title( disease.capitalize() + ' Reactions by Age')
    fig.tight_layout()

In [ ]:
#query_vid(merged_vaers_data_covid, 748142)
merged_vaers_data[merged_vaers_data['VAERS_ID']==748142]

In [ ]:
ana_disease(merged_vaers_data, 'THROMBOCYTOPENIA', 30) # THROMBOCYTOPENIA')

In [ ]:
ana_disease(merged_vaers_data, 'CANCER', 40)

In [ ]:
ana_disease(merged_vaers_data, 'MYOCARDITIS', 80)

In [ ]:
print("Cols NaN for all rows: ", cols_all_nan(merged_vaers_data)) 

In [ ]:
print(cols_with_nan(merged_vaers_data, 0.5))


## Missing value explaination
* RPT_DATE: Date the VAERS form was completed by reporter(bad system design, this could be easily get upon submit
* DIED/DATEDIED: Less than x% died


In [ ]:
df_patients_21["CUR_ILL"].replace(
    ["None", "No", "NONE", "unknown", "Unknown", "none", "no", "None known", "none known", 
     "None reported", "none reported", "UNKNOWN", "N/a", "None stated/Denied",
    "No other illness prior to vaccination or within the month prior", "NKDA", 
     "Individual was healthy prior to vaccination.", "None.", "UNK", "As noted above", "unsure", 
     "See item 12", "no acute illnesses", "No symptoms after COVID vaccinations", 
    "Na", "None Known", "No long term health problems reported on consent form",
    "no illnesses at time of vaccine", "non e", "nothing special."], 
    np.nan, inplace=True)
df_patients_21["CUR_ILL"].replace("Covid 19", "COVID_19", inplace=True)

In [ ]:
#pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20) # df_patients_21["CUR_ILL"].nunique())
print(df_patients_21["CUR_ILL"].nunique())
for i in df_patients_21["CUR_ILL"].unique():
    print(i)
    time.sleep(0.001)
    

In [ ]:
df_patients_21["CUR_ILL"].replace(
    ["None", "No", "NONE", "unknown", "Unknown", "none", "no", "None known", "none known", 
     "None reported", "none reported", "UNKNOWN", "N/a", "None stated/Denied",
    "No other illness prior to vaccination or within the month prior", "NKDA", 
     "Individual was healthy prior to vaccination.", "None.", "UNK", "As noted above", "unsure", 
     "See item 12", "no acute illnesses", "No symptoms after COVID vaccinations", 
    "Na"], 
    np.nan, inplace=True)
df_patients_21["CUR_ILL"].replace("Covid 19", "COVID_19", inplace=True)

## Reading vax data from https://covid.ourworldindata.org/data


In [ ]:
world_covid_data = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', parse_dates=['date'])

In [ ]:
us_covid_data = world_covid_data[world_covid_data['iso_code']=='USA']

In [ ]:
us_vax_data = pd.DataFrame()
us_vax_data[['date', 'new_vaccinations']] = us_covid_data[['date', 'new_vaccinations']] 
us_vax_data.fillna(0,inplace=True)
us_vax_data

In [ ]:
# Total US vax count
us_vax_data['new_vaccinations'].sum()

In [ ]:
# us_vax_data.groupby(us_vax_data['date'].sum().plot(figsize=FIGSIZE3,kind="bar"))
us_vax_data.query('new_vaccinations > 0')

In [ ]:
plt.figure(figsize=FIGSIZE3)
ax = V_PN['new_vaccinations'].plot(x='date', y='column1', linewidth=5, color='b')
ax.ticklabel_format(style='plain', axis='y')
plt.yticks(fontsize=16)
plt.xticks(fontsize=16)
plt.ylabel('Daily Vaccines Administered BLUE LINE', fontsize=16)
ax2 = ax.twinx()
plt.yticks(fontsize=16)
plt.title('VAERS report vs Vaccines Administered', fontsize=16)
plt.ylabel('Percent Reported +/- 99% RED LINE', fontsize=16)
ax2 = V_PN['percentage'].plot(x='date', y='column2', ax=ax2, color='r', linewidth=5)
plt.grid(None)
plt.show()

In [ ]:
VM_CDC_A = merged_vaers_data[['VAERS_ID', 'RECVDATE', 'STATE', 'AGE_YRS', 'CAGE_YR', 'CAGE_MO', 'SEX',
       'RPT_DATE', 'SYMPTOM_TEXT', 'DIED', 'DATEDIED', 'L_THREAT', 'ER_VISIT',
       'HOSPITAL', 'HOSPDAYS', 'X_STAY', 'DISABLE', 'RECOVD', 'VAX_DATE',
       'ONSET_DATE', 'NUMDAYS', 'LAB_DATA', 'V_ADMINBY', 'V_FUNDBY',
       'OTHER_MEDS', 'CUR_ILL', 'HISTORY', 'PRIOR_VAX', 'SPLTTYPE',
       'FORM_VERS', 'TODAYS_DATE', 'BIRTH_DEFECT', 'OFC_VISIT', 'ER_ED_VISIT',
       'ALLERGIES', 'VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES',
       'VAX_ROUTE', 'VAX_SITE', 'VAX_NAME', 'SYMPTOM1', 'SYMPTOMVERSION1',
       'SYMPTOM2', 'SYMPTOMVERSION2', 'SYMPTOM3', 'SYMPTOMVERSION3',
       'SYMPTOM4', 'SYMPTOMVERSION4', 'SYMPTOM5', 'SYMPTOMVERSION5']]
    

In [ ]:
VM_CDC_A.describe()

In [ ]:
RS = pd.DataFrame()
VM_CDC_A['VAERS']=int(1)
VM_CDC_A['WEEK'] = pd.to_datetime(VM_CDC_A['RECVDATE']) - pd.to_timedelta(7,unit='d')
pd.set_option('display.max_rows', 100)
print(VM_CDC_A[['VAERS_ID', 'RECVDATE', 'WEEK']])
VAERS = VM_CDC_A.groupby([pd.Grouper(key='WEEK', freq="W-MON")])['VAERS'].sum().reset_index().sort_values('WEEK')

In [ ]:
RS = pd.DataFrame()
VM_CDC_A['VAERS']=int(1)
VM_CDC_A['WEEK'] = pd.to_datetime(VM_CDC_A['RECVDATE']) - pd.to_timedelta(7,unit='d')
print(VM_CDC_A[['VAERS_ID', 'RECVDATE','WEEK']])
VAERS = VM_CDC_A.groupby([pd.Grouper(key='WEEK', freq="W-MON")])['VAERS'].sum().reset_index().sort_values('WEEK')

In [ ]:
VM_CDC_A[['VAERS', 'WEEK']]
VAERS

In [ ]:
pd.to_timedelta(7, unit='d')

In [ ]:
unique_vid_per_day = merged_vaers_data.groupby([merged_vaers_data['RECVDATE']])['VAERS_ID'].nunique()    
unique_vid_per_day